# Выбор локации для скважины

Добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

### Загрузка из изучение данных

In [2]:
data=[0,0,0]
data[0] = pd.read_csv('/datasets/geo_data_0.csv')
data[1] = pd.read_csv('/datasets/geo_data_1.csv')
data[2] = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
{d.shape for d in data}

{(100000, 5)}

In [4]:
[list(d.columns) for d in data]

[['id', 'f0', 'f1', 'f2', 'product'],
 ['id', 'f0', 'f1', 'f2', 'product'],
 ['id', 'f0', 'f1', 'f2', 'product']]

In [5]:
data[0].index

RangeIndex(start=0, stop=100000, step=1)

In [9]:
[data[i].duplicated().sum() for i in [0,1,2]]

[0, 0, 0]

In [6]:
for d in data:
    print(d.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

In [7]:
for d in data:
    print(d.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%       

### Разделение признаков 

In [8]:
features_train = [None]*len(data)
features_valid = [None]*len(data)
target_train = [None]*len(data)
target_valid = [None]*len(data)
for i in range(len(data)):
    f=data[i].drop(columns=['product','id'])
    t=data[i]['product']
    features_train[i], features_valid[i], target_train[i], target_valid[i] = train_test_split(
        f,t,test_size=0.25,random_state=12345)

### Маштабирование признаков 

In [9]:
for i in range(len(data)):
    scaler = StandardScaler()
    scaler.fit(features_train[i])
    features_train[i] = scaler.transform(features_train[i])
    features_valid[i] = scaler.transform(features_valid[i])

### Выводы

 В этом разделе мы загрузили, исследовали и подготовили данные. Исследование данных не выявило никаких аномалий (пропусков, не соответствий типов, дубликатов), но было обнаружено что числовые признаки имеют разный масштаб. Поэтому в подготовку данных вошло не только стандартная разбивка на тренировочные и валидационные данные, но и маштабирование (не целевых) числовых признаков.  

## Обучение и проверка модели

In [10]:
model = [None]*len(data)
prediction_valid = [None]*len(data)
rmse = [None]*len(data)
mean_product = [None]*len(data)
for i in range(len(data)):
    model[i]=LinearRegression()
    model[i].fit(features_train[i],target_train[i])
    prediction_valid[i] = model[i].predict(features_valid[i])
    rmse[i] = mean_squared_error(target_valid[i], prediction_valid[i], squared = False)
    mean_product[i] = prediction_valid[i].mean()
    print('Регион',i)
    print('Предсказания запаса сырья', mean_product[i])
    print('RMSE метрика', rmse[i],end='\n\n')

Регион 0
Предсказания запаса сырья 92.59256778438035
RMSE метрика 37.5794217150813

Регион 1
Предсказания запаса сырья 68.728546895446
RMSE метрика 0.893099286775617

Регион 2
Предсказания запаса сырья 94.96504596800489
RMSE метрика 40.02970873393434



Построенная модель на данных Регион 1 имеет наименьшее значение предсказания среднего запаса сырья в скважинах но точность модели существенно выше чем в регионе 0 и 2. 

## Подготовка к расчёту прибыли

In [11]:
number_to_explore =500
number_to_select = 200
revenue_per_barrel = 450
budget_well_dev=10**10
alpha_threshold=0.025

Достаточный объем сырья для безубыточной разработки новой скаважины составляет

In [12]:
min_oil_reserve = budget_well_dev/number_to_select/revenue_per_barrel
print(f'{min_oil_reserve:,.2f} баррель')

111,111.11 баррель


In [13]:
print('Среднии доходности скважин по регионам')
for i in range(len(data)):
    print(f'Регион {i} : {mean_product[i]*1000*revenue_per_barrel:,.2f}  руб.')

Среднии доходности скважин по регионам
Регион 0 : 41,666,655.50  руб.
Регион 1 : 30,927,846.10  руб.
Регион 2 : 42,734,270.69  руб.


Достаточный объем сырья, для безубыточной разработки,  привышает средний уровень объема сырья. Т.е. требуется отбор лучших скважин и проверка доходности такого отбора и риск убытка.

## Расчёт прибыли и рисков 

In [14]:
def revenue(target, predicted, count):
    probs_sorted = predicted.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return revenue_per_barrel *  1000 * selected.sum()  - budget_well_dev 

In [15]:
print(f'Прибыль на основе предсказаний {number_to_select} лучших скважин')
for i in range(len(data)):
    pred=pd.Series(data=prediction_valid[i],index=target_valid[i].index)
    print('Регион',i,end=':  ')
    print(f'{revenue(target_valid[i], pred, number_to_select):,.2f}')

Прибыль на основе предсказаний 200 лучших скважин
Регион 0:  3,320,826,043.14
Регион 1:  2,415,086,696.68
Регион 2:  2,710,349,963.60


## Расчет рисков и прибыли для каждого региона

In [18]:
state = np.random.RandomState(12345)
revenue_list=[None]*3
for i in range(len(data)):
    revenue_list[i]=[]
    for j in range(1000):
        prediction=pd.Series(data=prediction_valid[i],index=target_valid[i].index)
        target_subsample = target_valid[i].sample(n=500,replace=True,random_state=state)
        prediction_subsample = prediction[target_subsample.index]
        rev = revenue(target_subsample,prediction_subsample,200)
        revenue_list[i].append(rev)
        #subsample.quantile(0.99)
    values=pd.Series(revenue_list[i])
    neg=values<=0 #10**7
    p_value=neg.mean()
    lower = values.quantile(0.025) # < напишите код здесь >
    higher = values.quantile(0.975)
    #print(p_value,values.mean())
    print('Регион',i)
    print(f'   p-value для положительной доходсти = риск убытков = {p_value}')
    print(f'   Среднее значение прибыли  {values.mean():,.2f}' )  #- 10**7
    print(f'   95% интервал прибыли  ({lower:,.2f}, {higher:,.2f}) ')  #- 10**7
    print()

Регион 0
   p-value для положительной доходсти = риск убытков = 0.06
   Среднее значение прибыли  425,938,526.91
   95% интервал прибыли  (-102,090,094.84, 947,976,353.36) 

Регион 1
   p-value для положительной доходсти = риск убытков = 0.003
   Среднее значение прибыли  518,259,493.70
   95% интервал прибыли  (128,123,231.43, 953,612,982.07) 

Регион 2
   p-value для положительной доходсти = риск убытков = 0.062
   Среднее значение прибыли  420,194,005.34
   95% интервал прибыли  (-115,852,609.16, 989,629,939.84) 



## Выводы

Проведенно исследование данных по запасам скважин в трех регионах. Для каждого региона построена модель предсказания запаса. На основе построенных моделей проведен расчет риска убыточности и 95% доверительного интервала для прибыли. 

Не смотря на то, что среднее значение запасов сырья в регионе 1 самое низкое, среднее значение прибыли в этом регионе самое высокое. Также в регионе 1 самый низкий уровень риска убытко 0.3%, что ниже ниже порога 2.5%. В регионах 0 и 2 риск убытков состовляе прим. 6%.

Рекомендуем Регион 1 для разработки. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован